In [1]:
# !pip3 install gspread
# !pip3 install --upgrade google-api-python-client oauth2client 

In [9]:
import requests 
import json
import gspread
import os
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [10]:
def date_to_url(s):
    """ input 17/05/1996 
        return 19962F052F17 (url format)"""
    l = s.split('/')
    if len(l)==2:
        year=2022
        day,month = l
    elif len(l)==3:
        day,month,year = l

    return f'{year}%2F{month}%2F{day}'
def time_to_int(date):
    return int(date.split(':')[0])
def get_number_of_row(sh):
    return len(sh.col_values(1))

In [15]:
gc = gspread.service_account(filename=os.path.abspath('istheretgvmax-5273da7c9b7a.json')) #Put the path of your own json google token
sh = gc.open("is-there-tgvmax")
worksheet_list = [sh for sh in sh.worksheets() if '@' in sh.title]

sh=worksheet_list[0]
row_values = range(2,get_number_of_row(sh)+1)
for row_value in row_values:
    origine,destination,date,heure_min,heure_max,_,email_envoye,force_update=sh.row_values(row_value)
    print(origine,destination,date,heure_min,heure_max,_,email_envoye,force_update)
    request_start = 'https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card'
    full_request = f'{request_start}&refine.destination={destination}&refine.origine={origine}&refine.date={date_to_url(date)}'
    request=requests.get(full_request)
    dico = json.loads(request.text)
    voyages = dico['records']
    nb_trouve = 0
    for (i,voyage) in enumerate(voyages):
        heure_depart = time_to_int(voyage['fields']['heure_depart'])
        if heure_depart>=int(heure_min) and heure_depart<=int(heure_max):
            nb_trouve +=1
    sh.update(f'F{row_value}', nb_trouve)


PARIS+(intramuros) GRENOBLE 22/06 17 21 2 FALSE FALSE
GRENOBLE PARIS+(intramuros) 01/06 15 22 0 FALSE FALSE
BIARRITZ GRENOBLE 23/06 10 18 0 FALSE FALSE
PARIS+(intramuros) BORDEAUX+ST+JEAN 27/06 0 24  FALSE FALSE
